In [22]:
import pandas as pd
import numpy as np

%cd src
# Read the data from the text file
filename = '../data/TCGA_raw_58kgenes620samples.txt'
with open(filename, 'r') as f:
    lines = f.readlines()

# Extract row names from the first column
index = lines[0].split()[0:]

# Create DataFrames from the rest of the data
data = [line.split() for line in lines[1:]]
column_names = [row[0] for row in data]
data_values = np.matrix([row[1:] for row in data])

df = pd.DataFrame(data_values.transpose(), columns=column_names)
df.insert(0, 'Patient ID', index)
df

[Errno 2] No such file or directory: 'src'
/Users/ivamilojkovic/Breast-Cancer-Analysis/src


,Patient ID,5_8S_rRNA,5S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
0,TCGA-4N-A93T-01A-11R-A37K-07,0,6,0,31,75,996,3484,22,2,...,1039,155,756,1848,1,0,685,10621,2798,931
1,TCGA-4T-AA8H-01A-11R-A41B-07,0,1,0,1,12,876,1911,3,74,...,1559,74,451,1120,0,0,979,6497,4354,904
2,TCGA-5M-AAT4-01A-11R-A41B-07,0,5,2,1,10,820,5888,5,37,...,4130,234,1171,1896,2,0,1099,10500,2748,1582
3,TCGA-5M-AAT5-01A-21R-A41B-07,0,4,5,1,9,1359,5075,10,7,...,1849,124,830,1808,10,0,1208,8551,2253,1051
4,TCGA-5M-AAT6-01A-11R-A41B-07,0,3,2,6,32,21,11532,24,5,...,2261,122,469,1171,32,0,798,10001,2284,1316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,TCGA-QL-A97D-01A-12R-A41B-07,0,4,1,3,24,781,7087,5,2,...,3974,171,768,1870,0,0,1248,11030,4981,1519
616,TCGA-RU-A8FL-01A-11R-A37K-07,0,2,1,1,0,686,2369,4,1,...,4269,57,450,1755,2,0,1839,10272,1900,2401
617,TCGA-SS-A7HO-01A-21R-A37K-07,0,5,0,2,13,2298,8187,18,1,...,4289,182,642,3005,0,0,1426,4912,2297,1990
618,TCGA-T9-A92H-01A-11R-A37K-07,0,2,0,2,9,1761,5286,13,5,...,4472,204,1098,1249,0,0,1399,12393,2501,1527


In [23]:
%cd ..
from src.data_preprocessing import remove_extreme

# Raw counts processing
df_genes = df.iloc[:, 1:].astype('int')
df_after_gene_removal, samples_to_remove, feat_to_remove, feat_to_keep = remove_extreme(df_genes, change_X=True)
df_after_gene_removal.insert(loc=0, column='Patient ID', value=df['Patient ID'].values)

/Users/ivamilojkovic/Breast-Cancer-Analysis
There are 0 columns with more than 80% of Null values!
There are 24356 columns with more than 20% of count values greater than 4!


In [24]:
%cd src
df_with_dist = pd.read_excel(r'../data/13073_2023_1176_MOESM2_ESM.xlsx', engine='openpyxl', sheet_name='Table S2', header=2).drop(columns='Unnamed: 0')
dist_columns = ['distCRIS.A', 'distCRIS.B', 'distCRIS.C', 'distCRIS.D', 'distCRIS.E']
df_dist = df_with_dist[['aliquot_id'] + dist_columns]
df_dist = df_dist.rename(columns=
                         {'aliquot_id': 'Patient ID',
                          'distCRIS.A': 'CRIS.A',
                          'distCRIS.B': 'CRIS.B',
                          'distCRIS.C': 'CRIS.C',
                          'distCRIS.D': 'CRIS.D',
                          'distCRIS.E': 'CRIS.E',
                          })
dist_columns = [label.strip('dist') for label in dist_columns]
# Add subtype column (label corresponds to the highest value of correlation i.e. minimal of the distance)
df_dist['Subtype-from Parker centroids'] = df_dist[dist_columns].idxmin(axis=1)
df_dist

/Users/ivamilojkovic/Breast-Cancer-Analysis/src


,Patient ID,CRIS.A,CRIS.B,CRIS.C,CRIS.D,CRIS.E,Subtype-from Parker centroids
0,TCGA-4N-A93T-01A-11R-A37K-07,0.950423,1.023935,0.810985,1.097154,0.962194,CRIS.C
1,TCGA-4T-AA8H-01A-11R-A41B-07,0.891271,0.958085,1.019741,1.173642,1.105552,CRIS.A
2,TCGA-5M-AAT4-01A-11R-A41B-07,1.158505,0.991774,1.010710,1.035492,1.196177,CRIS.B
3,TCGA-5M-AAT5-01A-21R-A41B-07,1.151964,1.141232,1.149884,0.956858,1.118708,CRIS.D
4,TCGA-5M-AAT6-01A-11R-A41B-07,0.959272,0.818806,1.193775,0.959972,1.084572,CRIS.B
...,...,...,...,...,...,...,...
615,TCGA-QL-A97D-01A-12R-A41B-07,1.158747,1.053225,1.078349,1.241729,0.992571,CRIS.E
616,TCGA-RU-A8FL-01A-11R-A37K-07,1.251466,1.086701,0.882573,1.062215,0.942080,CRIS.C
617,TCGA-SS-A7HO-01A-21R-A37K-07,1.189990,1.107892,1.125795,0.568221,0.946591,CRIS.D
618,TCGA-T9-A92H-01A-11R-A37K-07,1.042662,1.068844,0.816676,1.122212,1.059630,CRIS.C


In [25]:
df_dist['Subtype-from Parker centroids'].value_counts(dropna=False)

CRIS.C    165
CRIS.A    160
CRIS.E    114
CRIS.D     98
CRIS.B     83
Name: Subtype-from Parker centroids, dtype: int64

In [26]:
# Merge DataFrames on the 'ID' column
merged_df = pd.merge(df_after_gene_removal, df_dist, on='Patient ID', how='left') 
merged_df

,Patient ID,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AAAS,AACS,...,ZYG11B,ZYX,ZZEF1,ZZZ3,CRIS.A,CRIS.B,CRIS.C,CRIS.D,CRIS.E,Subtype-from Parker centroids
0,TCGA-4N-A93T-01A-11R-A37K-07,31,75,996,3484,22,2,57,2400,2108,...,685,10621,2798,931,0.950423,1.023935,0.810985,1.097154,0.962194,CRIS.C
1,TCGA-4T-AA8H-01A-11R-A41B-07,1,12,876,1911,3,74,27,1663,2176,...,979,6497,4354,904,0.891271,0.958085,1.019741,1.173642,1.105552,CRIS.A
2,TCGA-5M-AAT4-01A-11R-A41B-07,1,10,820,5888,5,37,186,2845,3123,...,1099,10500,2748,1582,1.158505,0.991774,1.010710,1.035492,1.196177,CRIS.B
3,TCGA-5M-AAT5-01A-21R-A41B-07,1,9,1359,5075,10,7,89,2714,2031,...,1208,8551,2253,1051,1.151964,1.141232,1.149884,0.956858,1.118708,CRIS.D
4,TCGA-5M-AAT6-01A-11R-A41B-07,6,32,21,11532,24,5,282,1540,1546,...,798,10001,2284,1316,0.959272,0.818806,1.193775,0.959972,1.084572,CRIS.B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,TCGA-QL-A97D-01A-12R-A41B-07,3,24,781,7087,5,2,185,1928,3574,...,1248,11030,4981,1519,1.158747,1.053225,1.078349,1.241729,0.992571,CRIS.E
616,TCGA-RU-A8FL-01A-11R-A37K-07,1,0,686,2369,4,1,59,2402,2835,...,1839,10272,1900,2401,1.251466,1.086701,0.882573,1.062215,0.942080,CRIS.C
617,TCGA-SS-A7HO-01A-21R-A37K-07,2,13,2298,8187,18,1,300,5160,2863,...,1426,4912,2297,1990,1.189990,1.107892,1.125795,0.568221,0.946591,CRIS.D
618,TCGA-T9-A92H-01A-11R-A37K-07,2,9,1761,5286,13,5,148,2432,2966,...,1399,12393,2501,1527,1.042662,1.068844,0.816676,1.122212,1.059630,CRIS.C


In [27]:
# Subtract each value from 1 in the specified columns
merged_df[dist_columns] = 1 - merged_df[dist_columns]
merged_df

# max_dist = merged_df[dist_columns].max().max()

# # Subtract each value from maximum distance in the specified columns
# merged_df[dist_columns] = max_dist - merged_df[dist_columns]
# merged_df

,Patient ID,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AAAS,AACS,...,ZYG11B,ZYX,ZZEF1,ZZZ3,CRIS.A,CRIS.B,CRIS.C,CRIS.D,CRIS.E,Subtype-from Parker centroids
0,TCGA-4N-A93T-01A-11R-A37K-07,31,75,996,3484,22,2,57,2400,2108,...,685,10621,2798,931,0.049577,-0.023935,0.189015,-0.097154,0.037806,CRIS.C
1,TCGA-4T-AA8H-01A-11R-A41B-07,1,12,876,1911,3,74,27,1663,2176,...,979,6497,4354,904,0.108729,0.041915,-0.019741,-0.173642,-0.105552,CRIS.A
2,TCGA-5M-AAT4-01A-11R-A41B-07,1,10,820,5888,5,37,186,2845,3123,...,1099,10500,2748,1582,-0.158505,0.008226,-0.010710,-0.035492,-0.196177,CRIS.B
3,TCGA-5M-AAT5-01A-21R-A41B-07,1,9,1359,5075,10,7,89,2714,2031,...,1208,8551,2253,1051,-0.151964,-0.141232,-0.149884,0.043142,-0.118708,CRIS.D
4,TCGA-5M-AAT6-01A-11R-A41B-07,6,32,21,11532,24,5,282,1540,1546,...,798,10001,2284,1316,0.040728,0.181194,-0.193775,0.040028,-0.084572,CRIS.B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,TCGA-QL-A97D-01A-12R-A41B-07,3,24,781,7087,5,2,185,1928,3574,...,1248,11030,4981,1519,-0.158747,-0.053225,-0.078349,-0.241729,0.007429,CRIS.E
616,TCGA-RU-A8FL-01A-11R-A37K-07,1,0,686,2369,4,1,59,2402,2835,...,1839,10272,1900,2401,-0.251466,-0.086701,0.117427,-0.062215,0.057920,CRIS.C
617,TCGA-SS-A7HO-01A-21R-A37K-07,2,13,2298,8187,18,1,300,5160,2863,...,1426,4912,2297,1990,-0.189990,-0.107892,-0.125795,0.431779,0.053409,CRIS.D
618,TCGA-T9-A92H-01A-11R-A37K-07,2,9,1761,5286,13,5,148,2432,2966,...,1399,12393,2501,1527,-0.042662,-0.068844,0.183324,-0.122212,-0.059630,CRIS.C


In [28]:
import pickle

with open('../data/tcga_cris_raw_24356_620samples.pkl', 'wb') as file:
    pickle.dump(merged_df, file)